In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Rajesh Baidya
"""


'\nMinimal character-level Vanilla RNN model. Written by Rajesh Baidya\n'

In [2]:
import numpy as np
from pathlib import Path
import pickle

In [3]:
# data i/o
data = open('zafor-sir.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('Data has {} characters, {} unique.'.format(data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

Data has 969907 characters, 122 unique.


In [4]:
# hyper parameters
hidden_size = 100  # size of hidden layer of neurons
seq_length = 100  # number of steps to unroll the RNN for
learning_rate = 1e-1

In [5]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01  # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01  # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01  # hidden to output
bh = np.zeros((hidden_size, 1))  # hidden_bias
by = np.zeros((vocab_size, 1))  # output_bias

In [6]:
"""
inputs,targets are both list of integers.
hprev is Hx1 array of initial hidden state
returns the loss, gradients on model parameters, and last hidden state
"""
def lossFun(inputs, targets, hprev):   
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh)  # hidden state
        ys[t] = np.dot(Why, hs[t]) + by  # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))  # probabilities for next chars
        loss += -np.log(ps[t][targets[t], 0])  # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[
            t]] -= 1  # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext  # backpropagation into h
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)  # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]

In [7]:
"""
sample a sequence of integers from the model
h is memory state, seed_ix is seed letter for first time step
"""
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [8]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length  # loss at iteration 0

# load weights from saved model
weight_file = Path("weights/0.pkl")
if weight_file.is_file():
    # Getting back the Weights:
    with open('weights/0.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
        n, smooth_loss, loss, dWxh, dWhh, dWhy, dbh, dby, hprev = pickle.load(f)
        print("Resuming From %d iter with loss %f" % (n, smooth_loss))

        
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0  # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]
    # sample from the model now and then
    if n % 5000 == 0:
        sample_ix = sample(hprev, inputs[0], 500)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('iter %d, loss: %f' % (n, smooth_loss))
        with open("output/" + str(n // 50000) + ".txt", 'a+') as f:
            identifier = 'iter %d, loss: %f' % (n, smooth_loss)
            f.write(identifier)
            f.write('\n==================================\n')
            f.write(txt)
            f.write('\n\n')

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    if n % 500 == 0:
        print("Saving Weights on Iter: %f" % n)
        # Saving the Weights:
        with open('weights/0.pkl', 'wb') as f:
            pickle.dump([n, smooth_loss, loss, dWxh, dWhh, dWhy, dbh, dby, hprev], f)

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update

    p += seq_length  # move data pointer
    n += 1  # iteration counter

Resuming From 2500 iter with loss 253.912819
Saving Weights on Iter: 2500.000000
Saving Weights on Iter: 3000.000000


KeyboardInterrupt: 